In [94]:
import astropy.units as u
import numpy as np
from astroquery.simbad import Simbad
import astropy.coordinates as coord

from astropy.io import ascii
from astropy.io import fits
import os.path
# 
import elk
from elk.ensemble import EnsembleLC
from elk.lightcurve import BasicLightcurve

from astropy.table import Table, join, MaskedColumn, vstack, Column
from matplotlib import pyplot as plt

from astroML.datasets import fetch_sdss_galaxy_images, fetch_LINEAR_sample, fetch_LINEAR_geneva
from astroML.utils import split_samples

import matplotlib.pyplot as plt

from sklearn.metrics import ConfusionMatrixDisplay, PrecisionRecallDisplay
from sklearn.gaussian_process import GaussianProcessRegressor

from tqdm import tqdm
from scipy.special import entr

from astropy.table import Column
from scipy.stats import entropy as entr
import time

In [101]:
# gold_sample_clusters = Table.read('../cluster_summary_statistics.fits.fits')
f = open('cluster_names.txt', 'r')
names = f.readlines()
cnames = []
for i in names:
    t = i.split('\n')[0]
    cnames.append(t)
cnames

['hlsp_elk_tess_ffi_ascc-116_tess_v1_llc',
 'hlsp_elk_tess_ffi_loden-2045_tess_v1_llc',
 'hlsp_elk_tess_ffi_kmhk-955_tess_v1_llc',
 'hlsp_elk_tess_ffi_ngc-1776_tess_v1_llc',
 'hlsp_elk_tess_ffi_ngc-2003_tess_v1_llc',
 'hlsp_elk_tess_ffi_ngc-2133_tess_v1_llc',
 'hlsp_elk_tess_ffi_pir2016-vmc-29_tess_v1_llc',
 'hlsp_elk_tess_ffi_h88-336_tess_v1_llc',
 'hlsp_elk_tess_ffi_sl63-572_tess_v1_llc',
 'hlsp_elk_tess_ffi_kmhk-1292_tess_v1_llc',
 'hlsp_elk_tess_ffi_ngc-2095_tess_v1_llc',
 'hlsp_elk_tess_ffi_ngc-1863_tess_v1_llc',
 'hlsp_elk_tess_ffi_cl-lindsay-33_tess_v1_llc',
 'hlsp_elk_tess_ffi_ngc-6885_tess_v1_llc',
 'hlsp_elk_tess_ffi_h86-86_tess_v1_llc',
 'hlsp_elk_tess_ffi_fsr-0314_tess_v1_llc',
 'hlsp_elk_tess_ffi_sl63-152_tess_v1_llc',
 'hlsp_elk_tess_ffi_ngc-1902_tess_v1_llc',
 'hlsp_elk_tess_ffi_fsr-0253_tess_v1_llc',
 'hlsp_elk_tess_ffi_ngc-2127_tess_v1_llc',
 'hlsp_elk_tess_ffi_ogle-cl-smc-297_tess_v1_llc',
 'hlsp_elk_tess_ffi_bsdl-2934_tess_v1_llc',
 'hlsp_elk_tess_ffi_ngc-7031_tess_v

In [105]:
cluster_stats = Table.read('../cluster_summary_statistics.fits')

mw=Table.read(age_path + 'Use_MW.fits')

smc=Table.read(age_path + 'Bica_Cut_down.fits')

lmc=Table.read(age_path + 'Glatt_Cut_down.fits')

ages=[]
loc=[]
for name in list(cluster_stats['summary_fname']):
    for i in range(len(mw)):
        if mw[i]['NAME']== name:
            ages.append(mw[i]['LOG_AGE'])
            loc.append('MW')
    
    for j in range(len(smc)):
        if smc[j]['SimbadName']==name:
            ages.append(smc[j]['logAge'])
            loc.append('SMC')
            
    for k in range(len(lmc)):
        if lmc[k]['SimbadName']==name:
            ages.append(lmc[k]['Age'])
            loc.append('LMC')

cluster_stats['age'] = ages
cluster_stats.sort('n_rows')
list(cluster_stats['cluster_name'])

['hlsp_elk_tess_ffi_vdbergh-hagen-155_tess_v1_llc',
 'hlsp_elk_tess_ffi_turner-7_tess_v1_llc',
 'hlsp_elk_tess_ffi_teutsch-84_tess_v1_llc',
 'hlsp_elk_tess_ffi_ascc-9_tess_v1_llc',
 'hlsp_elk_tess_ffi_berkeley-43_tess_v1_llc',
 'hlsp_elk_tess_ffi_cl-kron-29_tess_v1_llc',
 'hlsp_elk_tess_ffi_cl-lindsay-56_tess_v1_llc',
 'hlsp_elk_tess_ffi_cl-lindsay-52_tess_v1_llc',
 'hlsp_elk_tess_ffi_cl-lindsay-65_tess_v1_llc',
 'hlsp_elk_tess_ffi_cl-kron-57_tess_v1_llc',
 'hlsp_elk_tess_ffi_cl-kron-55_tess_v1_llc',
 'hlsp_elk_tess_ffi_cl-kron-54_tess_v1_llc',
 'hlsp_elk_tess_ffi_cl-kron-47_tess_v1_llc',
 'hlsp_elk_tess_ffi_cl-kron-42_tess_v1_llc',
 'hlsp_elk_tess_ffi_cl-kron-34_tess_v1_llc',
 'hlsp_elk_tess_ffi_cl-kron-31_tess_v1_llc',
 'hlsp_elk_tess_ffi_cl-lindsay-51_tess_v1_llc',
 'hlsp_elk_tess_ffi_ngc-663_tess_v1_llc',
 'hlsp_elk_tess_ffi_ogle-cl-smc-65_tess_v1_llc',
 'hlsp_elk_tess_ffi_pir2016-vmc-26_tess_v1_llc',
 'hlsp_elk_tess_ffi_pir2016-vmc-36_tess_v1_llc',
 'hlsp_elk_tess_ffi_pismis-6_tes

In [72]:
path = '/uufs/astro.utah.edu/common/home/u1363702/notebooks/tess_clusters/TESS_Cluster_Age_ML'

In [73]:
use_full_sectors = True
# use_half_sectors = True
use_12day_rolling_window = False
use_stiched_sectors = False

In [76]:
import glob
path='/uufs/chpc.utah.edu/common/home/astro/zasowski/sinha/tess_data/light_curves/resampled_fits_tables/'
# filenames =  [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(path+"/")) for f in fn]
filenames = glob.glob(path + '*.fits')

In [77]:
def get_cluster_resampled_files(name):
    names = path + name + '*.fits'
    filenames = glob.glob(names)
    return filenames

In [78]:
fnames = get_cluster_resampled_files(cnames[0])

In [79]:
l_of_cs=[]
sectors = []
for file in fnames:
    hdul = fits.open(file)
    sectors.append(hdul[0].header['sectors'])
    hdul.close()
    data = elk.ensemble.from_fits(file)
    l_of_cs.append(elk.ensemble.from_fits(file))

In [80]:
l_of__all_lcs=[]
for i in range(len(l_of_cs)):
    data = []
    med = 0
    for j in range(len(l_of_cs[i].lcs)):
        lc = l_of_cs[i].lcs[j].corrected_lc
        if j == 0:
            med = np.nanmedian(lc['flux'])
        else:
            delta = np.nanmedian(lc['flux']) - med
            lc['flux'] = lc['flux'] - delta
        data.append(lc)
    stitched_data = vstack(data)
    l_of__all_lcs.append(stitched_data)

In [81]:
data_augmented_lcs=[]

for i in range(len(l_of__all_lcs)):

    if use_full_sectors:
        data_augmented_lcs.append(l_of__all_lcs[i])

In [82]:
names=[]

for i in range(len(l_of_cs)):
    j=0
    # while j < (len(l_of_cs[i].lcs)):

    if use_full_sectors:
        names.append(l_of_cs[i].callable)
        
        # if use_half_sectors:
        #     names.append(l_of_cs[i].callable) 
        #     names.append(l_of_cs[i].callable)
        
    j+=1
        
len(names)

15

In [83]:
%%time

l_of_stat_tables = []

import sys

# Start timer before the loop
start_time = time.time()

def print_progress_bar(i, total, length=30, start_time=None):
    percent = i / total
    filled = int(length * percent)
    bar = '█' * filled + '-' * (length - filled)

    elapsed = time.time() - start_time
    if i > 0:
        eta = (elapsed / i) * (total - i)
        eta_str = time.strftime("%H:%M:%S", time.gmtime(eta))
    else:
        eta_str = "--:--:--"

    sys.stdout.write(f'\rProgress: |{bar}| {round(percent * 100, 1)}% | ETA: {eta_str}')
    sys.stdout.flush()

for i in range(len(data_augmented_lcs)):
    print_progress_bar(i + 1, len(data_augmented_lcs), start_time=start_time)

    lc = BasicLightcurve(data_augmented_lcs[i]['time'],
                         data_augmented_lcs[i]['flux'],
                         data_augmented_lcs[i]['flux_err'],
                         sector=99)

    lc.get_stats_using_defaults()

    table = lc.get_stats_table(names[i])[[
        'name',
        'rms',
        'std',
        'MAD',
        'sigmaG',
        'skewness',
        'von_neumann_ratio',
        'J_Stetson',
        'max_power',
        'freq_at_max_power',
        'n_peaks',
        'ratio_of_power_at_high_v_low_freq',
        'FAP',
        'max_autocorrelation',
        'time_of_max_autocorrelation'
    ]]

    # Periodogram and frequency grid
    frequency_list = 1 / np.arange(0.04, 11, 0.01)
    periodogram = lc.periodogram

    # Because the lightcurves were smoothed over timescales >10days, don't use those scales.
    periodogram = periodogram[(frequency_list < 10)]
    frequency_list = frequency_list[(frequency_list < 10)]
    
    # Sum power in specific period bands
    sum_LSP_power_10_7_days = np.sum(periodogram[(frequency_list < 10) & (frequency_list > 7)])
    sum_LSP_power_7_4_days  = np.sum(periodogram[(frequency_list < 7) & (frequency_list > 4)])
    sum_LSP_power_4_1_days  = np.sum(periodogram[(frequency_list < 4) & (frequency_list > 1)])
    sum_LSP_power_1_p5_days = np.sum(periodogram[(frequency_list < 1) & (frequency_list > 0.5)])

    # Shannon entropy of the flux
    entropy_val = entr(data_augmented_lcs[i]['flux'].value).sum()

    # Add new features
    table.add_column(Column(sum_LSP_power_10_7_days), name='SumLSP_10_7_Day_Power')
    table.add_column(Column(sum_LSP_power_7_4_days), name='SumLSP_7_4_Day_Power')
    table.add_column(Column(sum_LSP_power_4_1_days), name='SumLSP_4_1_Day_Power')
    table.add_column(Column(sum_LSP_power_1_p5_days), name='SumLSP_1_p5_Day_Power')
    table.add_column(Column(entropy_val), name='Entropy')

    # NEW: add full periodogram as a variable-length array column
    table.add_column(Column([periodogram], name='FullPeriodogram'))

    l_of_stat_tables.append(table)


Progress: |██████████████████████████████| 100.0% | ETA: 00:00:00CPU times: user 35.6 s, sys: 3.61 ms, total: 35.6 s
Wall time: 19 s


In [84]:
stat_table=vstack(l_of_stat_tables)

In [85]:
stat_table['sectors'] = sectors



In [89]:
#get_ages
# path = '/Users/howard_willard/Desktop/TESS_Cluster_Age_ML-main'
age_path = '/uufs/chpc.utah.edu/common/home/astro/zasowski/sinha/tess_data/data/'

mw=Table.read(age_path + 'Use_MW.fits')

smc=Table.read(age_path + 'Bica_Cut_down.fits')

lmc=Table.read(age_path + 'Glatt_Cut_down.fits')

ages=[]
loc=[]
for name in list(stat_table['name']):
    for i in range(len(mw)):
        if mw[i]['NAME']== name:
            ages.append(mw[i]['LOG_AGE'])
            loc.append('MW')
    
    for j in range(len(smc)):
        if smc[j]['SimbadName']==name:
            ages.append(smc[j]['logAge'])
            loc.append('SMC')
            
    for k in range(len(lmc)):
        if lmc[k]['SimbadName']==name:
            ages.append(lmc[k]['Age'])
            loc.append('LMC')

In [90]:
stat_table['LOC'] = loc
stat_table['AGE'] = ages

In [92]:
#/uufs/chpc.utah.edu/common/home/astro/zasowski/sinha/tess_data/data/cluster_summary_files

def save_cluster_data(name, table):
    fpath = '/uufs/chpc.utah.edu/common/home/astro/zasowski/sinha/tess_data/data/cluster_summary_files/'
    f_name = '{}{}_summary_data.fits'.format(fpath,name)
    table.write(f_name,overwrite=True)

In [93]:
save_cluster_data(cnames[0], stat_table)